<a href="https://colab.research.google.com/github/vfrantc/dehaze_detect/blob/main/evaluate_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Nov  2 16:56:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


# RUN DETECTOR ON UNPROCESSED IMAGES

In [ ]:
!pip install mmcv-full
!rm -rf mmdetection
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -e .
!pip install Pillow==7.0.0

     |████████████████████████████████| 389 kB 7.6 MB/s 
     |████████████████████████████████| 185 kB 48.3 MB/s 
  Created wheel for mmcv-full: filename=mmcv_full-1.3.16-cp37-cp37m-linux_x86_64.whl size=43458578 sha256=742423e669c5f2f099f80942316bc8f5651ab76885630d90b945dfb1894c5040
  Stored in directory: /root/.cache/pip/wheels/ea/0a/1e/95428dfef725d66e6693b61180b928c293489b6d13df3ae0e2
Successfully built mmcv-full
Cloning into 'mmdetection'...
remote: Enumerating objects: 21546, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 21546 (delta 2), reused 0 (delta 0), pack-reused 21536
Receiving objects: 100% (21546/21546), 25.05 MiB | 13.71 MiB/s, done.
Resolving deltas: 100% (15129/15129), done.
/content/mmdetection
Obtaining file:///content/mmdetection
  Created wheel for terminaltables: filename=terminaltables-3.1.0-py3-none-any.whl size=15354 sha256=20becacb66a606f6cc429350a2727a1e010f74bd40a3a75795ccb0f84e3ab0a8
  S

In [ ]:
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth \
      -O checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth

--2021-11-02 17:24:55--  https://download.openmmlab.com/mmdetection/v2.0/scnet/scnet_x101_64x4d_fpn_20e_coco/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.78
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 611860651 (584M) [application/octet-stream]
Saving to: ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’

checkpoints/scnet_x 100%[===================>] 583.52M  9.86MB/s    in 64s     

2021-11-02 17:25:59 (9.17 MB/s) - ‘checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth’ saved [611860651/611860651]



In [ ]:
%cd mmdetection

/content/mmdetection


In [ ]:
from glob import glob
import os

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.datasets import build_dataloader
from mmdet.apis import single_gpu_test
from mmcv.parallel import MMDataParallel
from mmdet.apis import inference_detector, init_detector, show_result_pyplot

config = 'configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py'
checkpoint = 'checkpoints/scnet_x101_64x4d_fpn_20e_coco-fb09dec9.pth'
model = init_detector(config, checkpoint, device='cuda:0')
model = MMDataParallel(model, device_ids=[0])

Use load_from_local loader


In [ ]:
def create_dataset(folder):
  cfg = Config.fromfile('./configs/scnet/scnet_x101_64x4d_fpn_20e_coco.py')
  cfg.dataset_type = 'CocoDataset'
  PREFIX = folder
  cfg.data.test.img_prefix = folder
  cfg.data.test.ann_file = os.path.join(PREFIX, 'new_test.json')
  cfg.data.test.type = 'CocoDataset'
  dataset = build_dataset(cfg.data.test)
  #dataset.CLASSES = classes
  #dataset.cat_ids = list(range(len(dataset.CLASSES)))
  data_loader = build_dataloader(dataset, samples_per_gpu=1, workers_per_gpu=2, dist=False, shuffle=False)
  return dataset, data_loader

In [ ]:
!ls /content/drive/MyDrive/hazy_detect

/content/drive/MyDrive/hazy_detect


In [ ]:
#!cp /content/drive/MyDrive/hazy_detect2/out_gca.zip .
#!cp /content/drive/MyDrive/hazy_detect2/dcp_coco.zip . 
!cp /content/drive/MyDrive/hazy_detect2/out_refined.zip .
#!cp /content/drive/MyDrive/hazy_detect2/dehazenet_coco.zip .
#!cp /content/drive/MyDrive/hazy_detect2/ide_coco.zip .

In [ ]:
#!unzip -q out_gca.zip
#!unzip -q dcp_coco.zip
!unzip -q out_refined.zip
#!unzip -q dehazenet_coco.zip
#!unzip -q ide_coco.zip

### gca

In [ ]:
method = 'out_gca'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
dataset.cat_ids.append(91)
metric = dataset.evaluate(outputs, metric='bbox')

!zip -r detected_gca.zip out_out_gca/
!cp -r detected_gca.zip /content/drive/MyDrive/hazy_detect

mkdir: cannot create directory ‘out_out_gca’: File exists
loading annotations into memory...
Done (t=0.43s)
creating index...
index created!
[                                                  ] 0/4320, elapsed: 0s, ETA:

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.4 task/s, elapsed: 3076s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.68s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=82.39s).
Accumulating evaluation results...
DONE (t=10.35s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.368
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.576
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.396
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.156
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.350
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.489
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.450
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.450
 Average Recall     (AR) @[ IoU=0.50:0.95 

In [ ]:
!cp -r detected_gca.zip /content/drive/MyDrive/

### dcp


In [ ]:
method = 'dcp_coco'
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.46s)
creating index...
index created!
[                                                  ] 0/4320, elapsed: 0s, ETA:

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.4 task/s, elapsed: 3073s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=7.66s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=77.66s).
Accumulating evaluation results...
DONE (t=10.37s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.360
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.568
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.387
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.339
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.487
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.438
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.438
 Average Recall     (AR) @[ IoU=0.50:0.95 

In [ ]:
!zip -r detected_dcp.zip out_dcp_coco/
!cp -r detected_dcp.zip /content/drive/MyDrive/

updating: out_dcp_coco/ (stored 0%)
updating: out_dcp_coco/JPEGImages/ (stored 0%)
updating: out_dcp_coco/JPEGImages/HEB_Google_618.png (deflated 2%)
updating: out_dcp_coco/JPEGImages/WST_Bing_521.png (deflated 2%)
updating: out_dcp_coco/JPEGImages/BJ_Baidu_748.png (deflated 4%)
updating: out_dcp_coco/JPEGImages/SY_Google_321.png (deflated 14%)
updating: out_dcp_coco/JPEGImages/JL_Baidu_057.png (deflated 7%)
updating: out_dcp_coco/JPEGImages/JL_Google_674.png (deflated 6%)
updating: out_dcp_coco/JPEGImages/SH_Google_161.png (deflated 3%)
updating: out_dcp_coco/JPEGImages/QD_Baidu_169.png (deflated 2%)
updating: out_dcp_coco/JPEGImages/SY_Google_44.png (deflated 1%)
updating: out_dcp_coco/JPEGImages/MLS_Bing_108.png (deflated 4%)
updating: out_dcp_coco/JPEGImages/GSGL_Baidu_607.png (deflated 1%)
updating: out_dcp_coco/JPEGImages/hv42_675.png (deflated 2%)
updating: out_dcp_coco/JPEGImages/BL_Baidu_213.png (deflated 2%)
updating: out_dcp_coco/JPEGImages/hv30_145.png (deflated 3%)
updatin

### Refined

In [ ]:
!cp /content/drive/MyDrive/new_test.json out_refined

In [ ]:
!ls out_refined/JPEGImages

In [ ]:
method = 'out_refined'
!rm -rf out_out_refined
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

loading annotations into memory...
Done (t=0.40s)
creating index...
index created!
[                                                  ] 0/4320, elapsed: 0s, ETA:

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4320/4320, 1.5 task/s, elapsed: 2960s, ETA:     0s
Evaluating bbox...
Loading and preparing results...
DONE (t=0.60s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=77.36s).
Accumulating evaluation results...
DONE (t=9.41s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.359
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.571
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.383
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.145
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.486
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.442
 Average Recall     (AR) @[ IoU=0.50:0.95 |

In [ ]:
!zip -r detected_refined.zip out_out_refined/
!cp -r detected_refined.zip /content/drive/MyDrive/

  adding: out_out_refined/ (stored 0%)
  adding: out_out_refined/JPEGImages/ (stored 0%)
  adding: out_out_refined/JPEGImages/XG_Baidu_053.png (deflated 11%)
  adding: out_out_refined/JPEGImages/BL_Bing_186.png (deflated 4%)
  adding: out_out_refined/JPEGImages/OLD_Google_216.png (deflated 10%)
  adding: out_out_refined/JPEGImages/GSGL_Baidu_078.png (deflated 5%)
  adding: out_out_refined/JPEGImages/hv12_759.png (deflated 2%)
  adding: out_out_refined/JPEGImages/LSJ_Baidu_900.png (deflated 4%)
  adding: out_out_refined/JPEGImages/PL_Google_202.png (deflated 2%)
  adding: out_out_refined/JPEGImages/HF_Bing_627.png (deflated 3%)
  adding: out_out_refined/JPEGImages/GSGL_Baidu_448.png (deflated 6%)
  adding: out_out_refined/JPEGImages/hv36_630.png (deflated 4%)
  adding: out_out_refined/JPEGImages/QD_Bing_851.png (deflated 2%)
  adding: out_out_refined/JPEGImages/hv40_412.png (deflated 4%)
  adding: out_out_refined/JPEGImages/hv39_614.png (deflated 2%)
  adding: out_out_refined/JPEGImages

# Dehazenet

In [ ]:
method = 'dehazenet_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!zip -r detected_dehazenet.zip out_dehazenet/
!cp -r detected_dehazenet.zip /content/drive/MyDrive/hazy_detect

# IDE

In [ ]:
method = 'dide_coco'
!cp /content/drive/MyDrive/new_test.json dide_coco
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!zip -r detected_ide.zip out_ide_coco/
!cp -r detected_ide.zip /content/drive/MyDrive/hazy_detect

# RTTS

In [ ]:
!rm -rf rtts_coco
!cp /content/mmdetection/drive/MyDrive/haze/data/RTTS.zip .
!unzip -q RTTS.zip
!mv RTTS rtts_coco 
!cp /content/mmdetection/drive/MyDrive/new_test.json rtts_coco

In [ ]:
method = 'rtts_coco'
!mkdir out_{method}
dataset, data_loader = create_dataset(f'./{method}')
dataset.cat_ids.append(91)
outputs = single_gpu_test(model, data_loader, True, f'./out_{method}', 0.3)
metric = dataset.evaluate(outputs, metric='bbox')

In [ ]:
!zip -r detected_rtts_coco.zip out_rtts_coco/
!cp -r detected_rtts_coco.zip /content/drive/MyDrive/hazy_detect